In [ ]:
import ast
import datetime as dt
import json
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'
sys.path.insert(0, '..')

import gps

### initialize the cache from previous csv files

In [ ]:
files = [os.path.join('gmaps_cache', f) for f in os.listdir('gmaps_cache')]

dfs = []
for f in files:
    a = pd.read_csv(f)

    a['dtRetrieved'] = dt.datetime.now().date
    a['radius']  = 50
    a['rankby']  = 'prominence'
    a['source']  = 'Google Places'
    a.lat = np.round(a.lat, 5)
    a.lon = np.round(a.lon, 5)
    
    dfs.append(a)
    
df = pd.concat(dfs, sort=True)
df['mask_'] = [not x for x in df.loc[:, ['lat', 'lon']].duplicated(keep='first')]
df.head()

In [ ]:
records = [
    gps.PlaceRequest().from_tuple(t, gps.ApiSource.GMAPS) 
    for t in df.loc[df.mask_].itertuples()
]

for r in tqdm(records):
    c = ast.literal_eval(r.content)
    c = json.dumps(c)
    r.content = c

In [ ]:
for r in tqdm(records):
    with gps.session_scope() as session:
        session.add(r)

### example recall the entire cache and parse the content

In [ ]:
with gps.session_scope() as s:
    t = pd.concat([ti.dataframe for ti in s.query(gps.PlaceRequest).all()], sort=True)

cats = pd.DataFrame([gps.parse_gmap_types(c) for c in t.content])
cats.loc[cats.rank_order != -1].head(10)

### make some requests to ensure the cache is responding correctly

In [ ]:
request = [
    gps.PlaceRequest(lat=r.lat, lon=r.lon, source=gps.ApiSource.FROM_NAME(r.source), radius=r.radius)
    for r in t.iloc[np.random.choice(np.arange(len(t)), 10, replace=False)].itertuples()
]

In [ ]:
gps.request_nearby_places(request)['request']